# AI CHATBOT

<hr>

In this jupyter notebook, we will build a chatbot using sequence to sequence model.

In [39]:
# Import the libraries
import numpy as np
import tensorflow as tf
import contractions
from tqdm import tqdm
import re
import time

<br>

# 1. Load the Dataset

<hr>

For creating our chatbot, we will use a data set called <a href="https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html">Cornell Movie Dialogs Corpus</a> which this corpus contains a large metadata-rich collection of fictional conversations extracted from raw movie scripts:

- 220,579 conversational exchanges between 10,292 pairs of movie characters
- involves 9,035 characters from 617 movies
- in total 304,713 utterances


However for our purpose, we will only use <i>movie_conversations.txt</i> and <i>movie_lines.txt</i>.


In [2]:
# Load the lines dataset
lines = open(file = "./dataset/movie_lines.txt", encoding = "utf-8", errors = "ignore").read()

# Split the dataset for every line
lines = lines.split("\n")

In [3]:
# Print lines
lines[:5]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]

In lines dataset, we have 5 columns. Now let's consider the first line:

- **Line id** --> like L1045
- **User id** --> like u0
- **Movie id** --> like m0
- **User name** --> like BIANCA
- **Text** --> like They do not!

In [4]:
# Load the conversations dataset
conversations = open(file = "./dataset/movie_conversations.txt", encoding = "utf-8", errors = "ignore").read()

# Split the dataset for every line
conversations = conversations.split("\n")

In [5]:
# Print convesations
conversations[:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

In conversations dataset, we have 4 columns. Now let's consider the first line:

- **User number 1** --> like u0
- **User number 2** --> like u2
- **Movie id** --> like m0
- **Conversations (array of line ids)** --> ['L194', 'L195', 'L196', 'L197']

<br>

# 2. Data Prepration

<hr>

Before applying the preprocessing steps, we need to prepare our dataset. In other word, we need to exactly specify our inputs and our outputs.

In [6]:
### Create a dictionary that maps each line id and its line text

# Initialize an empty dictionary
id2line = {}

# Iterate through lines
for i_line in lines:
    
    # Split the line with +++$+++                       # _ before lines means that this is temporary
    _lines = i_line.split(" +++$+++ ")
    
    # Make sure the length is 5 (otherwise we are going to get the wrong information because of indexing)
    if len(_lines) == 5:
        
        # Get the line id 
        line_id = _lines[0]
        
        # Get the text
        text = _lines[-1]
        
        # Map line ids to its text
        id2line[line_id] = text

In [7]:
# Take a look at id2line
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [8]:
### Create a list of all conversations

# Initialize an empty array
conversation_ids = []

# Iterate through conversations                  # We don't take the last row because it's an empty string
for i_conversation in conversations[:-1]:
    
    # Split each conversation with +++$+++
    _conversation = i_conversation.split(" +++$+++ ")
    
    # Get the lines ids
    _conversation = _conversation[-1]
    
    # Remove the square brackets                 # Remmber that this array is inside a string so we can use indexing like below
    _conversation = _conversation[1:-1]
    
    # Remove quotes
    _conversation = _conversation.replace("'", "")
    
    # Remove empty spaces
    _conversation = _conversation.replace(" ", "")
    
    # Split the conversation with ,
    _conversation = _conversation.split(",")
    
    # Append the conversations to our list
    conversation_ids.append(_conversation)

In [9]:
# Print conversation_ids
conversation_ids

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

In [10]:
### Get the questions and answers seperatly

# Initialize an empry list for questions
questions = []

# Initialize an empry list for answers
answers = []

# Iterate through conversation ids
for i_conversation in conversation_ids:
    
    # Iterate through length of i_conversation
    for index in range(len(i_conversation) - 1):

        # Get the text of current index
        current_text = id2line[i_conversation[index]]
        
        # Append the text to questions array
        questions.append(current_text)
        
        # Get the text of next index
        next_text = id2line[i_conversation[index + 1]]
        
        # Append the text to answers array
        answers.append(next_text)

In [11]:
# Take a look at questions
questions[:5]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---"]

In [12]:
# Take a look at answers
answers[:5]

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.']

<br>

# 3. Data Preprocessing

<hr>

In this section we will take the following steps for preprocessing our dataset:
1. Lowercasing the text
2. Decontracting the text
3. Remove the punctuation

In [13]:
# Preprocessing function
def preprocess_text(text):
    """
    Function for applying the preprocesssing steps.
    """
    # 1. Lowercase the text
    text = text.lower()
    
    # 2. De-contract the text
    text = contractions.fix(text)
    
    # 3. Remove the punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    return text

In [14]:
# Apply preprocesing function to our questions
questions_preprocessed = []
for i_question in tqdm(questions):
    questions_preprocessed.append(preprocess_text(i_question))

# Apply preprocesing function to our answers
answers_preprocessed = []
for i_answer in tqdm(answers):
    answers_preprocessed.append(preprocess_text(i_answer))

100%|██████████| 221616/221616 [00:06<00:00, 36477.82it/s]


In [15]:
# Take a look at preprocessed questions
questions_preprocessed[:5]

['can we make this quick   roxanne korrine and andrew barrett are having an incredibly horrendous public break  up on the quad   again ',
 'well  i thought we would start with pronunciation  if that is okay with you ',
 'not the hacking and gagging and spitting part   please ',
 'you are asking me out   that is so cute  what is your name again ',
 'no  no  it is my fault    we did not have a proper introduction    ']

In [16]:
# Take a look at preprocessed answers
answers_preprocessed[:5]

['well  i thought we would start with pronunciation  if that is okay with you ',
 'not the hacking and gagging and spitting part   please ',
 'okay    then how  bout we try out some french cuisine   saturday   night ',
 'forget it ',
 'cameron ']

In [17]:
# Initialize an empty dictionary for counting words
word2count = {}

# Create a word count dictionary for questions
for i_question in tqdm(questions_preprocessed):
    for i_word in i_question.split():
        if i_word not in word2count:
            word2count[i_word] = 1
        else:
            word2count[i_word] += 1

# Create a word count dictionary for answers
for i_answer in tqdm(questions_preprocessed):
    for i_word in i_answer.split():
        if i_word not in word2count:
            word2count[i_word] = 1
        else:
            word2count[i_word] += 1

100%|██████████| 221616/221616 [00:01<00:00, 138908.15it/s]


In [18]:
### Remove the least used words

# Threshold; If occurrence of each word is below this number, it will get deleted
threshold = 20    

# Map the question words to a unique integer
questionwords2int = {}
word_number = 0
for i_word, i_count in word2count.items():
    if i_count >= threshold:
        questionwords2int[i_word] = word_number
        word_number += 1

# Map the answer words to a unique integer
answerwords2int = {}
word_number = 0
for i_word, i_count in word2count.items():
    if i_count >= threshold:
        answerwords2int[i_word] = word_number
        word_number += 1

In [19]:
# Take a look at questionsword2int
questionwords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'barrett': 7,
 'are': 8,
 'having': 9,
 'an': 10,
 'incredibly': 11,
 'public': 12,
 'break': 13,
 'up': 14,
 'on': 15,
 'the': 16,
 'again': 17,
 'well': 18,
 'i': 19,
 'thought': 20,
 'would': 21,
 'start': 22,
 'with': 23,
 'if': 24,
 'that': 25,
 'is': 26,
 'okay': 27,
 'you': 28,
 'not': 29,
 'part': 30,
 'please': 31,
 'asking': 32,
 'me': 33,
 'out': 34,
 'so': 35,
 'cute': 36,
 'what': 37,
 'your': 38,
 'name': 39,
 'no': 40,
 'it': 41,
 'my': 42,
 'fault': 43,
 'did': 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 'introduction': 48,
 'cameron': 49,
 'thing': 50,
 'I': 51,
 'am': 52,
 'at': 53,
 'mercy': 54,
 'of': 55,
 'particularly': 56,
 'hideous': 57,
 'breed': 58,
 'loser': 59,
 'sister': 60,
 'date': 61,
 'until': 62,
 'she': 63,
 'does': 64,
 'why': 65,
 'mystery': 66,
 'used': 67,
 'to': 68,
 'be': 69,
 'really': 70,
 'popular': 71,
 'when': 72,
 'started': 73,
 'high': 74,
 'school': 75,
 

In [20]:
# Take a look at answerwords2int
answerwords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'barrett': 7,
 'are': 8,
 'having': 9,
 'an': 10,
 'incredibly': 11,
 'public': 12,
 'break': 13,
 'up': 14,
 'on': 15,
 'the': 16,
 'again': 17,
 'well': 18,
 'i': 19,
 'thought': 20,
 'would': 21,
 'start': 22,
 'with': 23,
 'if': 24,
 'that': 25,
 'is': 26,
 'okay': 27,
 'you': 28,
 'not': 29,
 'part': 30,
 'please': 31,
 'asking': 32,
 'me': 33,
 'out': 34,
 'so': 35,
 'cute': 36,
 'what': 37,
 'your': 38,
 'name': 39,
 'no': 40,
 'it': 41,
 'my': 42,
 'fault': 43,
 'did': 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 'introduction': 48,
 'cameron': 49,
 'thing': 50,
 'I': 51,
 'am': 52,
 'at': 53,
 'mercy': 54,
 'of': 55,
 'particularly': 56,
 'hideous': 57,
 'breed': 58,
 'loser': 59,
 'sister': 60,
 'date': 61,
 'until': 62,
 'she': 63,
 'does': 64,
 'why': 65,
 'mystery': 66,
 'used': 67,
 'to': 68,
 'be': 69,
 'really': 70,
 'popular': 71,
 'when': 72,
 'started': 73,
 'high': 74,
 'school': 75,
 

These are some special tokens which is used in seq2seq:

- `SOS` - Start of string which is the first token that should start in the decoding layers.
    
- `EOS` - End of sentence. As soon as decoder generates this token we consider the answer to be complete (you can't use usual punctuation marks for this purpose cause their meaning can be different)

- `UNK` - Unknown token. This is used to replace the rare words that did not fit in your vocabulary. So your sentence My name is guotong1988 will be translated into My name is _unk_.

- `PAD` - Your GPU (or CPU at worst) processes your training data in batches and all the sequences in your batch should have the same length. If the max length of your sequence is 8, your sentence My name is guotong1988 will be padded from either side to fit this length: My name is guotong1988 _pad_ _pad_ _pad_ _pad_

- `OUT` - Tokens by which all the words that were filtered out by our previous dictionaries will be replaced. We call it OUT as in filter out.

In [21]:
# Adding the special tokens to the previous two dictionaries
tokens = ["<PAD>", "<EOS>", "<OUT>", "<SOS>"]

for i_token in tokens:
    questionwords2int[i_token] = len(questionwords2int) + 1
    
for i_token in tokens:
    answerwords2int[i_token] = len(answerwords2int) + 1

In [22]:
# Create the inverse dictionary of answerwords2int
answerints2word = {i_index: i_word for i_word, i_index in answerwords2int.items()}

In [23]:
# Take a look at answerints2word
answerints2word

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'quick',
 5: 'and',
 6: 'andrew',
 7: 'barrett',
 8: 'are',
 9: 'having',
 10: 'an',
 11: 'incredibly',
 12: 'public',
 13: 'break',
 14: 'up',
 15: 'on',
 16: 'the',
 17: 'again',
 18: 'well',
 19: 'i',
 20: 'thought',
 21: 'would',
 22: 'start',
 23: 'with',
 24: 'if',
 25: 'that',
 26: 'is',
 27: 'okay',
 28: 'you',
 29: 'not',
 30: 'part',
 31: 'please',
 32: 'asking',
 33: 'me',
 34: 'out',
 35: 'so',
 36: 'cute',
 37: 'what',
 38: 'your',
 39: 'name',
 40: 'no',
 41: 'it',
 42: 'my',
 43: 'fault',
 44: 'did',
 45: 'have',
 46: 'a',
 47: 'proper',
 48: 'introduction',
 49: 'cameron',
 50: 'thing',
 51: 'I',
 52: 'am',
 53: 'at',
 54: 'mercy',
 55: 'of',
 56: 'particularly',
 57: 'hideous',
 58: 'breed',
 59: 'loser',
 60: 'sister',
 61: 'date',
 62: 'until',
 63: 'she',
 64: 'does',
 65: 'why',
 66: 'mystery',
 67: 'used',
 68: 'to',
 69: 'be',
 70: 'really',
 71: 'popular',
 72: 'when',
 73: 'started',
 74: 'high',
 75: 'school',
 

In [24]:
# Add <EOS> to the end of each answers;  <EOS> is used for decoding part of our seq2seq which we start the process of decoding after recieving <EOS>
for index in range(len(answers_preprocessed)):
    answers_preprocessed[index] += " <EOS>"

In [25]:
# Take a look at answers_preprocessed
answers_preprocessed[:5]

['well  i thought we would start with pronunciation  if that is okay with you  <EOS>',
 'not the hacking and gagging and spitting part   please  <EOS>',
 'okay    then how  bout we try out some french cuisine   saturday   night  <EOS>',
 'forget it  <EOS>',
 'cameron  <EOS>']

In [26]:
# Translate all the questions & answers into integer
# And replace all the words that were filtered out by <OUT>
questions_to_int = []
for i_question in tqdm(questions_preprocessed):
    ints = []
    for i_word in i_question.split():
        if i_word not in questionwords2int:
            ints.append(questionwords2int["<OUT>"])
        else:
            ints.append(questionwords2int[i_word])
    questions_to_int.append(ints)
            
answers_to_int = []
for i_answer in tqdm(answers_preprocessed):
    ints = []
    for i_word in i_answer.split():
        if i_word not in answerwords2int:
            ints.append(answerwords2int["<OUT>"])
        else:
            ints.append(answerwords2int[i_word])
    answers_to_int.append(ints)

100%|██████████| 221616/221616 [00:02<00:00, 97719.07it/s] 


In [27]:
# Take a look at questions_to_int
questions_to_int[:5]

[[0,
  1,
  2,
  3,
  4,
  8794,
  8794,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  8794,
  12,
  13,
  14,
  15,
  16,
  8794,
  17],
 [18, 19, 20, 1, 21, 22, 23, 8794, 24, 25, 26, 27, 23, 28],
 [29, 16, 8794, 5, 8794, 5, 8794, 30, 31],
 [28, 8, 32, 33, 34, 25, 26, 35, 36, 37, 26, 38, 39, 17],
 [40, 40, 41, 26, 42, 43, 1, 44, 29, 45, 46, 47, 48]]

In [28]:
# Take a look at answers_to_int
answers_to_int[:5]

[[18, 19, 20, 1, 21, 22, 23, 8794, 24, 25, 26, 27, 23, 28, 8793],
 [29, 16, 8794, 5, 8794, 5, 8794, 30, 31, 8793],
 [27, 76, 100, 1511, 1, 872, 34, 488, 389, 8794, 213, 245, 8793],
 [248, 41, 8793],
 [49, 8793]]

In [29]:
# Sort the questions & answers by the length of questions
# And remove the long texts (longer than 25 words)
sorted_clean_questions = []
sorted_clean_answers = []

for i_length in range(1, 25 + 1):
    for i in enumerate(questions_to_int):
        if len(i[1]) == i_length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_to_int[i[0]])

In [30]:
# Take a look at sorted_clean_questions; At start the lenfth is 1 and it get larger at the end
sorted_clean_questions

[[49],
 [65],
 [126],
 [150],
 [136],
 [40],
 [178],
 [40],
 [183],
 [184],
 [224],
 [37],
 [65],
 [136],
 [65],
 [113],
 [134],
 [297],
 [214],
 [37],
 [225],
 [183],
 [27],
 [65],
 [183],
 [8794],
 [461],
 [253],
 [183],
 [193],
 [8794],
 [8794],
 [676],
 [100],
 [8794],
 [8794],
 [37],
 [40],
 [37],
 [8794],
 [150],
 [93],
 [37],
 [37],
 [8794],
 [643],
 [40],
 [40],
 [948],
 [834],
 [1118],
 [40],
 [234],
 [72],
 [214],
 [40],
 [143],
 [1252],
 [214],
 [1110],
 [1110],
 [1110],
 [1110],
 [214],
 [337],
 [152],
 [27],
 [95],
 [676],
 [37],
 [1136],
 [1245],
 [1485],
 [1064],
 [1491],
 [214],
 [1530],
 [37],
 [37],
 [214],
 [1578],
 [1578],
 [1578],
 [1578],
 [1578],
 [1578],
 [27],
 [676],
 [70],
 [676],
 [234],
 [100],
 [1608],
 [1578],
 [1578],
 [1578],
 [8794],
 [1578],
 [1578],
 [70],
 [1646],
 [681],
 [1769],
 [214],
 [39],
 [214],
 [1211],
 [214],
 [1330],
 [214],
 [18],
 [126],
 [37],
 [65],
 [1818],
 [1749],
 [214],
 [1826],
 [214],
 [214],
 [1799],
 [225],
 [1144],
 [72],
 

In [31]:
# Take a look at sorted_clean_answers
sorted_clean_answers

[[16,
  50,
  26,
  49,
  51,
  52,
  53,
  16,
  54,
  55,
  46,
  56,
  57,
  58,
  55,
  59,
  42,
  60,
  19,
  0,
  29,
  61,
  62,
  63,
  64,
  8793],
 [8794,
  66,
  63,
  67,
  68,
  69,
  70,
  71,
  72,
  63,
  73,
  74,
  75,
  76,
  41,
  77,
  78,
  79,
  63,
  80,
  81,
  55,
  41,
  82,
  83,
  8793],
 [105, 8793],
 [1508, 79, 104, 1527, 34, 152, 615, 8793],
 [28, 156, 230, 3, 6310, 8793],
 [27, 28, 8, 163, 68, 256, 68, 1265, 100, 68, 621, 8793],
 [288, 742, 135, 8793],
 [28, 242, 98, 68, 194, 229, 41, 8793],
 [196, 8793],
 [21, 28, 127, 619, 33, 46, 1492, 49, 8793],
 [42,
  2550,
  160,
  51,
  45,
  80,
  46,
  106,
  1109,
  53,
  972,
  16,
  8794,
  147,
  222,
  524,
  8793],
 [114, 8794, 96, 46, 273, 8793],
 [181, 77, 79, 46, 274, 275, 8793],
 [281,
  25,
  19,
  282,
  51,
  21,
  113,
  97,
  283,
  78,
  119,
  280,
  270,
  77,
  267,
  41,
  5,
  19,
  45,
  29,
  284,
  285,
  96,
  8794,
  212,
  155,
  5,
  42,
  8794,
  8794,
  8794,
  286,
  8793],
 [11

<br>

# 4. Building the Seq2Seq Model

---

In [56]:
# Function for creating the placeholers for inputs and targets
def model_inputs():
    """
    Create placeholders for inputs, targets, learning rate, and keep probability for dropout.
    """
    # Inputs
    inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'input')
    
    # Targets
    targets = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'target')
    
    # Learning rate
    lr = tf.placeholder(dtype = tf.float32, name = 'learning_rate')
    
    # Keep probability (for dropout)
    keep_prob = tf.placeholder(dtype = tf.float32, name = 'keep_prob')
    
    return inputs, targets, lr, keep_prob

In [57]:
# Function for preprocesing the targets (create batches + add <SOS> at the start of each sentence)
def preprocess_targets(targets, word2int, batch_size):
    """
    Preprocessing the target by:
        1. Creating batches
        2. Adding <SOS> at the begining of each sentence
    """
    # Left side: <SOS> tensor
    left_side = tf.fill(dims = [batch_size, 1], value = word2int['<SOS>'])
    
    # Right side: Sliced targets
    right_side = tf.strided_slice(input_ = targets, begin = [0,0], end = [batch_size, -1], strides = [1,1])
    
    # Concatenating two tensors
    preprocessed_targets = tf.concat(values = [left_side, right_side], axis = 1)
    
    return preprocessed_targets

In [58]:
# Create the Encoder RNN Layer
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    """
    Encoder RNN Layer
    """
    # Basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units = rnn_size)
    
    # Add dropout
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(cell = lstm, input_keep_prob = keep_prob)
    
    # Create RNN cell composed sequentially of multiple simple cells
    encoder_cell = tf.contrib.rnn.MultiRNNCell(cells = [lstm_dropout] * num_layers)
    
    # Dynamic version of Bidirectional RNN
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state

In [59]:
# Decode the training set
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, 
                        output_function, keep_prob, batch_size):
    """
    Decoding the training set.
    """
    # Initialize the attention
    attention_states = tf.zeros(shape = [batch_size, 1, decoder_cell.output_size])
    
    # Prepare keys / values / functions for attention
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states = attention_states, 
                                                                                                                                    attention_option = "bahdanau", 
                                                                                                                                    num_units = decoder_cell.output_size)
    
    # Attentional decoder function for dynamic_rnn_decoder during training
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state = encoder_state[0],
                                                                              attention_keys = attention_keys,
                                                                              attention_values = attention_values,
                                                                              attention_score_fn = attention_score_function,
                                                                              attention_construct_fn = attention_construct_function,
                                                                              name = "attn_dec_train")
    
    # Dynamic RNN decoder for a sequence-to-sequence model specified by RNNCell and decoder function
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(cell = decoder_cell,
                                                                                                              decoder_fn = training_decoder_function,
                                                                                                              inputs = decoder_embedded_input,
                                                                                                              sequence_length = sequence_length,
                                                                                                              scope = decoding_scope)
    # Add dropout
    decoder_output_dropout = tf.nn.dropout(x = decoder_output, keep_prob = keep_prob)
    
    return output_function(decoder_output_dropout)

In [60]:
# Decode the test/validation set
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, 
                    decoding_scope, output_function, keep_prob, batch_size):
    """
    Decoding the test set / validation set.
    """
    # Initialize the attention
    attention_states = tf.zeros(shape = [batch_size, 1, decoder_cell.output_size])
    
    # Prepare keys / values / functions for attention
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states = attention_states, 
                                                                                                                                    attention_option = "bahdanau", 
                                                                                                                                    num_units = decoder_cell.output_size)
    # Attentional decoder function for dynamic_rnn_decoder during inference
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn = output_function,
                                                                              encoder_state = encoder_state[0],
                                                                              attention_keys = attention_keys,
                                                                              attention_values = attention_values,
                                                                              attention_score_fn = attention_score_function,
                                                                              attention_construct_fn = attention_construct_function,
                                                                              embeddings = decoder_embeddings_matrix,
                                                                              start_of_sequence_id = sos_id,
                                                                              end_of_sequence_id = eos_id,
                                                                              maximum_length = maximum_length,
                                                                              num_decoder_symbols = num_words,
                                                                              name = "attn_dec_inf")
    
    # Dynamic RNN decoder for a sequence-to-sequence model specified by RNNCell and decoder function
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(cell = decoder_cell,
                                                                                                                decoder_fn = test_decoder_function,
                                                                                                                scope = decoding_scope)
    return test_predictions

In [61]:
# Create Decoder RNN
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, 
                rnn_size, num_layers, word2int, keep_prob, batch_size):
    """
    Creating Decoder RNN
    """
    # Variable scope
    with tf.variable_scope("decoding") as decoding_scope:
        
        # Basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        
        # Add dropout
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        
        # Create RNN cell composed sequentially of multiple simple cells
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        
        # Initialize the weights
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        
        # Initialize the biases
        biases = tf.zeros_initializer()
        
        # Add a Fully Connected Layer
        output_function = lambda x: tf.contrib.layers.fully_connected(inputs = x,
                                                                      num_outputs = num_words,
                                                                      normalizer_fn = None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        # Decoding the training set
        training_predictions = decode_training_set(encoder_state = encoder_state,
                                                   decoder_cell = decoder_cell,
                                                   decoder_embedded_input = decoder_embedded_input,
                                                   sequence_length = sequence_length,
                                                   decoding_scope = decoding_scope,
                                                   output_function = output_function,
                                                   keep_prob = keep_prob,
                                                   batch_size = batch_size)
        
        # Reuse variables
        decoding_scope.reuse_variables()
        
        # Decoding the test set
        test_predictions = decode_test_set(encoder_state = encoder_state,
                                           decoder_cell = decoder_cell,
                                           decoder_embeddings_matrix = decoder_embeddings_matrix,
                                           sos_id = word2int['<SOS>'],
                                           eos_id = word2int['<EOS>'],
                                           maximum_length = sequence_length - 1,
                                           num_words = num_words,
                                           decoding_scope = decoding_scope,
                                           output_function = output_function,
                                           keep_prob = keep_prob,
                                           batch_size = batch_size)
        
    return training_predictions, test_predictions

In [62]:
# Build the seq2seq model
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, 
                  encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    """
    Seq2Seq Model
    """
    # Maps a sequence of symbols to a sequence of embeddings
    encoder_embedded_input = tf.contrib.layers.embed_sequence(ids = inputs,
                                                              vocab_size = answers_num_words + 1,
                                                              embed_dim = encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    
    # Encoder RNN Layer
    encoder_state = encoder_rnn(rnn_inputs = encoder_embedded_input, 
                                rnn_size = rnn_size, 
                                num_layers = num_layers, 
                                keep_prob = keep_prob, 
                                sequence_length = sequence_length)
    
    # Preprocessing the target by creating batches & Adding <SOS> at the begining of each sentence
    preprocessed_targets = preprocess_targets(targets = targets, 
                                              word2int = questionswords2int, 
                                              batch_size = batch_size)
    
    # Decoder embedding matrix
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform(shape = [questions_num_words + 1, decoder_embedding_size], minval = 0, maxval = 1))
    
    # Decoder embedding input
    decoder_embedded_input = tf.nn.embedding_lookup(params = decoder_embeddings_matrix, 
                                                    ids = preprocessed_targets)
    
    # Decoder RNN Layer
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input = decoder_embedded_input,
                                                         decoder_embeddings_matrix = decoder_embeddings_matrix,
                                                         encoder_state = encoder_state,
                                                         num_words = questions_num_words,
                                                         sequence_length = sequence_length,
                                                         rnn_size = rnn_size,
                                                         num_layers = num_layers,
                                                         word2int = questionswords2int,
                                                         keep_prob = keep_prob,
                                                         batch_size = batch_size)
    
    return training_predictions, test_predictions